# Model Metrics

We've put together our first Q&A model, and explored some of the metrics we can use to measure Q&A performance. In this notebook we're going to merge both of these and measure our Q&A model performance on the SQuAD 2.0 validation set as a whole.

First, we load our SQuAD validation data.

In [6]:
import json

with open('../../data/squad/dev.json', 'r') as f:
    squad = json.load(f)

# we will limit it to the first 100 samples in the interest of time
squad = squad[:100]

Next, let's setup the QA pipeline again using the `deepset/bert-base-cased-squad2` model.

In [3]:
from transformers import BertTokenizer, BertForQuestionAnswering, pipeline

modelname = 'deepset/bert-base-cased-squad2'

tokenizer = BertTokenizer.from_pretrained(modelname)
model = BertForQuestionAnswering.from_pretrained(modelname)

qa = pipeline('question-answering', model=model, tokenizer=tokenizer)

And now we build a list of predicted answers `model_out` and true answers `reference` and calculate the ROUGE score based on these.

In [4]:
from tqdm import tqdm

model_out = []
reference = []

for pair in tqdm(squad, leave=True):
    ans = qa({
        'question': pair['question'],
        'context': pair['context']
    })
    # append the prediction and reference to the respective lists
    model_out.append(ans['answer'])
    reference.append(pair['answer'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:57<00:00,  1.73it/s]


This make take some time to process. The processing speed of our models will improve as we begin using more efficient implementations over the next few sections.

Once that has finished processing, we can calculate our ROUGE scores just like we did before.

In [5]:
from rouge import Rouge

# initialize
rouge = Rouge()

# get scores
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.550452380952381,
  'p': 0.5426666666666666,
  'f': 0.5268095208535719},
 'rouge-2': {'r': 0.27141666666666664,
  'p': 0.2693181818181818,
  'f': 0.2580314670722219},
 'rouge-l': {'r': 0.550452380952381,
  'p': 0.5426666666666666,
  'f': 0.5268095208535719}}

That doesn't seem to be scoring as high as we would expect, if we print some of the results we can see why:

In [7]:
# recalculate individual scores
scores = rouge.get_scores(model_out, reference)

print(model_out[4], ' | ', reference[4], ' | ', scores[4]['rouge-1']['f'])
print(model_out[22], ' | ', reference[22], ' | ', scores[22]['rouge-1']['f'])

Rollo,  |  Rollo  |  0.0
north  |  the north  |  0.6666666622222223


Clearly the punctuation differences are causing our ROUGE score to view these words as not matching. To fix this, we'll import `re` and remove any characters that are not spaces, letters, or numbers.

In [8]:
import re

clean = re.compile('(?i)[^0-9a-z ]')

# apply this to both lists
model_out = [clean.sub('', text) for text in model_out]
reference = [clean.sub('', text) for text in reference]

In [9]:
# recalculate individual scores
scores = rouge.get_scores(model_out, reference)

print(model_out[4], ' | ', reference[4], ' | ', scores[4]['rouge-1']['f'])
print(model_out[22], ' | ', reference[22], ' | ', scores[22]['rouge-1']['f'])

Rollo  |  Rollo  |  0.999999995
north  |  the north  |  0.6666666622222223


These scores are looking better now, let's calculate the average again:

In [10]:
rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.7429523809523809,
  'p': 0.7693333333333334,
  'f': 0.7236666627070026},
 'rouge-2': {'r': 0.3114166666666667,
  'p': 0.3093181818181818,
  'f': 0.2980314669222219},
 'rouge-l': {'r': 0.7429523809523809,
  'p': 0.7693333333333334,
  'f': 0.7236666627070026}}

Now we are seeing much more realistic scores